In [0]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
from glob import glob

In [0]:
base_model = MobileNet(input_shape=None, alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=True, weights='imagenet', input_tensor=None, 
                       pooling=None, classes=1000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
17227776/17225924 [==============================] - 1s 0us/step


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/chest_xray.zip') as zip_ref:
  zip_ref.extractall('')

In [0]:
import os
os.listdir()

['.config', 'chest_xray', '__MACOSX', 'drive', 'sample_data']

In [0]:
IMAGE_SIZE=[224, 224]
train_data_dir = 'chest_xray/train'
validation_data_dir = 'chest_xray/test'

In [0]:
folders = glob('chest_xray/train/*')

In [0]:
for layer in base_model.layers[-10:]:
  layer.trainable = True

In [0]:
BATCH_SIZE = 32

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                    target_size=IMAGE_SIZE, 
                                                    batch_size=BATCH_SIZE)
test_set = test_datagen.flow_from_directory(validation_data_dir,
                                            target_size = (224, 224),
                                            batch_size = BATCH_SIZE,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [0]:
x = base_model.output
x = Flatten()(x)

In [0]:
prediction = Dense(len(folders), activation='softmax')(x)

In [0]:
model = Model(inputs=base_model.input, outputs=prediction)

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [0]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'],
)

In [0]:
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

In [0]:
%%time
r = model.fit_generator(
  train_generator,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=16,
  validation_steps=len(test_set)
)

Epoch 1/10
16/16 [==============================] - 28s 2s/step - loss: 0.6503 - acc: 0.8164 - val_loss: 0.6735 - val_acc: 0.6250
Epoch 2/10
16/16 [==============================] - 16s 1s/step - loss: 0.6220 - acc: 0.8730 - val_loss: 0.6696 - val_acc: 0.6250
Epoch 3/10
16/16 [==============================] - 16s 984ms/step - loss: 0.6122 - acc: 0.8301 - val_loss: 0.6655 - val_acc: 0.6282
Epoch 4/10
16/16 [==============================] - 16s 990ms/step - loss: 0.5875 - acc: 0.8750 - val_loss: 0.6648 - val_acc: 0.6250
Epoch 5/10
16/16 [==============================] - 15s 967ms/step - loss: 0.5683 - acc: 0.8789 - val_loss: 0.6603 - val_acc: 0.6250
Epoch 6/10
16/16 [==============================] - 16s 990ms/step - loss: 0.5616 - acc: 0.8594 - val_loss: 0.6625 - val_acc: 0.6250
Epoch 7/10
16/16 [==============================] - 15s 967ms/step - loss: 0.5428 - acc: 0.8828 - val_loss: 0.6585 - val_acc: 0.6362
Epoch 8/10
16/16 [==============================] - 15s 967ms/step - loss: 

In [0]:
model.save('/content/drive/My Drive/Colab Notebooks/MobileNet.h5') 

In [0]:
from tensorflow.keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [0]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/Colab Notebooks/MobileNet.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[<tf.Tensor 'dense_1/Softmax:0' shape=(?, 2) dtype=float32>]
[<tf.Tensor 'input_1_1:0' shape=(?, 224, 224, 3) dtype=float32>]


In [0]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 628 variables.
INFO:tensorflow:Converted 628 variables to const ops.


In [0]:
tf.train.write_graph(frozen_graph, "model", "tf_model.pb", as_text = False)

'model/tf_model.pb'

In [0]:
import tensorflow as tf
from tensorflow.python.platform import gfile
sess = tf.Session()
f = gfile.FastGFile("./model/tf_model.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()
sess.graph.as_default()
tf.import_graph_def(graph_def)

In [0]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
converter = trt.TrtGraphConverter(
	input_graph_def=graph_def,
	nodes_blacklist=['dense_1/Softmax:0'])
frozen_graph = converter.convert()

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0


In [0]:
tf.train.write_graph(frozen_graph, 'mobilenet','tftrt_model.pb',as_text = False)

'mobilenet/tftrt_model.pb'

Inferencing 

In [0]:
all_nodes=len([1 for n in frozen_graph.node])
print("no. of all_nodes in frozen graph: ",all_nodes)
trt_engine_nodes = len([1 for n in graph_def.node if str(n.op)=='TRTeng'])
print("no. of trt_engine_nodes in tensorrt graph: ",trt_engine_nodes)
all_nodes=len([1 for n in graph_def.node])
print("no. of all_nodes in tensorrt graph: ",all_nodes)

no. of all_nodes in frozen graph:  273
no. of trt_engine_nodes in tensorrt graph:  0
no. of all_nodes in tensorrt graph:  894


In [0]:
import os
files = os.listdir('chest_xray/test/PNEUMONIA/')
img1 = image.load_img(r'chest_xray/test/PNEUMONIA/' + files[0], target_size=(224, 224))
img_array1 = image.img_to_array(img1)
img_array_expanded_dims1 = np.expand_dims(img_array1, axis=0)
input_img = preprocess_input(img_array_expanded_dims1)

for i in files[1:70]:
    img2 = image.load_img('chest_xray/test/PNEUMONIA/' + i, target_size=(224, 224))
    img_array2 = image.img_to_array(img2)
    img_array_expanded_dims2 = np.expand_dims(img_array2, axis=0)
    img2 = preprocess_input(img_array_expanded_dims2)

    input_img = np.concatenate((input_img, img2),axis=0)

In [0]:
# function to read a ".pb" model 
# (can be used to read frozen model or TensorRT model)
def read_pb_graph(model):
  with gfile.FastGFile(model,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def

In [0]:
TENSORRT_MODEL_PATH = "/content/mobilenet/tftrt_model.pb"
import time
import tensorflow as tf
graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name('input_1_1:0')
        output = sess.graph.get_tensor_by_name('dense_1/Softmax:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred_ = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT = total_time / n_time_inference
        print("average inference time : ", avg_time_tensorRT)

needed time in inference-0:  0.049783945083618164
needed time in inference-1:  0.04974484443664551
needed time in inference-2:  0.04976511001586914
needed time in inference-3:  0.04928088188171387
needed time in inference-4:  0.04957437515258789
needed time in inference-5:  0.05356597900390625
needed time in inference-6:  0.048813819885253906
needed time in inference-7:  0.05054783821105957
needed time in inference-8:  0.04915571212768555
needed time in inference-9:  0.04896712303161621
needed time in inference-10:  0.05098247528076172
needed time in inference-11:  0.04954886436462402
needed time in inference-12:  0.04892325401306152
needed time in inference-13:  0.04913616180419922
needed time in inference-14:  0.04891371726989746
needed time in inference-15:  0.05291318893432617
needed time in inference-16:  0.04901695251464844
needed time in inference-17:  0.048601627349853516
needed time in inference-18:  0.04888749122619629
needed time in inference-19:  0.04852700233459473
needed 